In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import names
from collections import defaultdict
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/apple/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [3]:
# newsgroups.data

In [4]:
data = pd.Series(newsgroups.data)

In [5]:
df = pd.DataFrame(data)

In [6]:
df.columns = ['Text'] + df.columns.tolist()[1:]

In [7]:
df['Target'] = pd.Series(newsgroups.target)

In [8]:
df

,Text,Target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [9]:
df['Target'].nunique()

20

In [10]:
df['Target'].isnull().value_counts()

False    18846
Name: Target, dtype: int64

In [11]:
# email_tokenizer = RegexpTokenizer(r'\S+@\S+(?:\.\S+)+')
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')


In [12]:
df 

,Text,Target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [13]:
# common_names = names.words()



In [14]:
# df['Text'] = df['Text'].apply(lambda x: [item for item in x if item not in common_names])




In [15]:
df['Text'] = df['Text'].apply(lambda row: re.sub(r"\S*@\S*\s?","",row))

In [16]:
df

,Text,Target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: (David Nye)\nDN> A neurology\nDN> co...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [19]:
df

,Text,Target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: (David Nye)\nDN> A neurology\nDN> co...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [22]:
df['Text'] = df['Text'].str.lower()

In [23]:
df

,Text,Target
0,\n\ni am sure some bashers of pens fans are pr...,10
1,my brother is in the market for a high-perform...,3
2,\n\n\n\n\tfinally you said what you dream abou...,17
3,\nthink!\n\nit's the scsi card doing the dma t...,3
4,1) i have an old jasmine drive which i cann...,4
...,...,...
18841,dn> from: (david nye)\ndn> a neurology\ndn> co...,13
18842,\nnot in isolated ground recepticles (usually ...,12
18843,i just installed a dx2-66 cpu in a clone mothe...,3
18844,\nwouldn't this require a hyper-sphere. in 3-...,1


In [24]:
df['tokenized_text'] = df.apply(lambda row: tokenizer.tokenize(row['Text']),axis=1)

In [25]:
df

,Text,Target,tokenized_text
0,\n\ni am sure some bashers of pens fans are pr...,10,"[i, am, sure, some, bashers, of, pens, fans, a..."
1,my brother is in the market for a high-perform...,3,"[my, brother, is, in, the, market, for, a, hig..."
2,\n\n\n\n\tfinally you said what you dream abou...,17,"[finally, you, said, what, you, dream, about, ..."
3,\nthink!\n\nit's the scsi card doing the dma t...,3,"[think, it, s, the, scsi, card, doing, the, dm..."
4,1) i have an old jasmine drive which i cann...,4,"[1, i, have, an, old, jasmine, drive, which, i..."
...,...,...,...
18841,dn> from: (david nye)\ndn> a neurology\ndn> co...,13,"[dn, from, david, nye, dn, a, neurology, dn, c..."
18842,\nnot in isolated ground recepticles (usually ...,12,"[not, in, isolated, ground, recepticles, usual..."
18843,i just installed a dx2-66 cpu in a clone mothe...,3,"[i, just, installed, a, dx2, 66, cpu, in, a, c..."
18844,\nwouldn't this require a hyper-sphere. in 3-...,1,"[wouldn, t, this, require, a, hyper, sphere, i..."


In [26]:
stop_words=set(stopwords.words("english"))

In [27]:
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [item for item in x if item not in stop_words])



In [28]:
df

,Text,Target,tokenized_text
0,\n\ni am sure some bashers of pens fans are pr...,10,"[sure, bashers, pens, fans, pretty, confused, ..."
1,my brother is in the market for a high-perform...,3,"[brother, market, high, performance, video, ca..."
2,\n\n\n\n\tfinally you said what you dream abou...,17,"[finally, said, dream, mediterranean, new, are..."
3,\nthink!\n\nit's the scsi card doing the dma t...,3,"[think, scsi, card, dma, transfers, disks, scs..."
4,1) i have an old jasmine drive which i cann...,4,"[1, old, jasmine, drive, cannot, use, new, sys..."
...,...,...,...
18841,dn> from: (david nye)\ndn> a neurology\ndn> co...,13,"[dn, david, nye, dn, neurology, dn, consultati..."
18842,\nnot in isolated ground recepticles (usually ...,12,"[isolated, ground, recepticles, usually, unusu..."
18843,i just installed a dx2-66 cpu in a clone mothe...,3,"[installed, dx2, 66, cpu, clone, motherboard, ..."
18844,\nwouldn't this require a hyper-sphere. in 3-...,1,"[require, hyper, sphere, 3, space, 4, points, ..."


In [29]:
lemmatizer = WordNetLemmatizer()

In [30]:
df['lemmatized'] = df['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

In [31]:
df

,Text,Target,tokenized_text,lemmatized
0,\n\ni am sure some bashers of pens fans are pr...,10,"[sure, bashers, pens, fans, pretty, confused, ...","[sure, bashers, pen, fan, pretty, confused, la..."
1,my brother is in the market for a high-perform...,3,"[brother, market, high, performance, video, ca...","[brother, market, high, performance, video, ca..."
2,\n\n\n\n\tfinally you said what you dream abou...,17,"[finally, said, dream, mediterranean, new, are...","[finally, said, dream, mediterranean, new, are..."
3,\nthink!\n\nit's the scsi card doing the dma t...,3,"[think, scsi, card, dma, transfers, disks, scs...","[think, scsi, card, dma, transfer, disk, scsi,..."
4,1) i have an old jasmine drive which i cann...,4,"[1, old, jasmine, drive, cannot, use, new, sys...","[1, old, jasmine, drive, cannot, use, new, sys..."
...,...,...,...,...
18841,dn> from: (david nye)\ndn> a neurology\ndn> co...,13,"[dn, david, nye, dn, neurology, dn, consultati...","[dn, david, nye, dn, neurology, dn, consultati..."
18842,\nnot in isolated ground recepticles (usually ...,12,"[isolated, ground, recepticles, usually, unusu...","[isolated, ground, recepticles, usually, unusu..."
18843,i just installed a dx2-66 cpu in a clone mothe...,3,"[installed, dx2, 66, cpu, clone, motherboard, ...","[installed, dx2, 66, cpu, clone, motherboard, ..."
18844,\nwouldn't this require a hyper-sphere. in 3-...,1,"[require, hyper, sphere, 3, space, 4, points, ...","[require, hyper, sphere, 3, space, 4, point, s..."


In [32]:
df.lemmatized = df.lemmatized.apply(lambda x: " ".join(x) )

In [33]:
df

,Text,Target,tokenized_text,lemmatized
0,\n\ni am sure some bashers of pens fans are pr...,10,"[sure, bashers, pens, fans, pretty, confused, ...",sure bashers pen fan pretty confused lack kind...
1,my brother is in the market for a high-perform...,3,"[brother, market, high, performance, video, ca...",brother market high performance video card sup...
2,\n\n\n\n\tfinally you said what you dream abou...,17,"[finally, said, dream, mediterranean, new, are...",finally said dream mediterranean new area grea...
3,\nthink!\n\nit's the scsi card doing the dma t...,3,"[think, scsi, card, dma, transfers, disks, scs...",think scsi card dma transfer disk scsi card dm...
4,1) i have an old jasmine drive which i cann...,4,"[1, old, jasmine, drive, cannot, use, new, sys...",1 old jasmine drive cannot use new system unde...
...,...,...,...,...
18841,dn> from: (david nye)\ndn> a neurology\ndn> co...,13,"[dn, david, nye, dn, neurology, dn, consultati...",dn david nye dn neurology dn consultation chea...
18842,\nnot in isolated ground recepticles (usually ...,12,"[isolated, ground, recepticles, usually, unusu...",isolated ground recepticles usually unusual co...
18843,i just installed a dx2-66 cpu in a clone mothe...,3,"[installed, dx2, 66, cpu, clone, motherboard, ...",installed dx2 66 cpu clone motherboard tried m...
18844,\nwouldn't this require a hyper-sphere. in 3-...,1,"[require, hyper, sphere, 3, space, 4, points, ...",require hyper sphere 3 space 4 point specifies...


In [34]:
X,y = df.lemmatized,df.Target

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [36]:
str(X[0])

'sure bashers pen fan pretty confused lack kind post recent pen massacre devil actually bit puzzled bit relieved however going put end non pittsburghers relief bit praise pen man killing devil worse thought jagr showed much better regular season stats also lot fo fun watch playoff bowman let jagr lot fun next couple game since pen going beat pulp jersey anyway disappointed see islander lose final regular season game pen rule'

In [64]:
count_vectorizer = CountVectorizer(stop_words=None,min_df=0.01)

In [70]:
count_train = count_vectorizer.fit_transform(X_train)

In [71]:
count_train

<14134x1220 sparse matrix of type '<class 'numpy.int64'>'
	with 475621 stored elements in Compressed Sparse Row format>

In [72]:
df_BOW = pd.DataFrame(count_train.toarray(), columns = count_vectorizer.get_feature_names())
df_BOW

,00,000,10,100,11,12,13,14,15,16,...,writing,written,wrong,wrote,yeah,year,yes,yet,york,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
df_BOW.columns

Index(['00', '000', '10', '100', '11', '12', '13', '14', '15', '16',
       ...
       'writing', 'written', 'wrong', 'wrote', 'yeah', 'year', 'yes', 'yet',
       'york', 'young'],
      dtype='object', length=1220)

In [74]:
df_BOW['00'].sum()

1777

In [75]:
df_BOW.drop([col for col, val in df_BOW.sum().iteritems() if val < 10], axis=1, inplace=True)


In [76]:
df_BOW

,00,000,10,100,11,12,13,14,15,16,...,writing,written,wrong,wrote,yeah,year,yes,yet,york,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [77]:
count_test = count_vectorizer.transform(X_test)
count_test

<4712x1220 sparse matrix of type '<class 'numpy.int64'>'
	with 161419 stored elements in Compressed Sparse Row format>

In [78]:
df_BOW_test = pd.DataFrame(count_test.toarray(), columns = count_vectorizer.get_feature_names())
df_BOW_test

,00,000,10,100,11,12,13,14,15,16,...,writing,written,wrong,wrote,yeah,year,yes,yet,york,young
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4710,0,0,1,0,2,1,1,1,2,1,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df['Target'].value_counts()

10    999
15    997
8     996
9     994
11    991
13    990
7     990
5     988
14    987
2     985
12    984
3     982
6     975
1     973
4     963
17    940
16    910
0     799
18    775
19    628
Name: Target, dtype: int64

In [48]:
df['Text'][18844
          ]

# dataframe.str.replace(r"\S*@\S*\s?")

"\nwouldn't this require a hyper-sphere.  in 3-space, 4 points over specifies\na sphere as far as i can see.  unless that is you can prove that a point\nexists in 3-space that is equi-distant from the 4 points, and this may not\nnecessarily happen.\n\ncorrect me if i'm wrong (which i quite possibly am!)\n\nsteve\n---\n\n\n"

In [49]:
df1 = pd.Series(newsgroups.data)

In [50]:
# df1[18845].str.replace(r"\S*@\S*\s?")

In [51]:
df2 = df1.to_frame()

In [52]:
import re

In [53]:
test = re.sub(r"\S*@\S*\s?","",df2[0][18845])

In [54]:
test

'After a tip from Gary Crum I got on the Phone\nwith "Pontiac Systems" or "Pontaic Customer Service" or whatever, and\ninquired about a rumoured Production Hold on the Formula Firebird and\nTrans Am.  BTW, Talking with the dealer I bought the car from got me\nnowhere.  After being routed to a "Firebird Specialist", I was able\nto confirm that this is in fact the case.\n\nAt first, there was some problem with the 3:23 performance axle ratio.\nShe wouldn\'t go into any details, so I don\'t know if there were some\nshipped that had problems, or if production was held up because they\nsimply didn\'t have the proper parts from the supplier.  As I say, she\nwas pretty vague on that, so if anyone else knows anything about this,\nfeel free to respond.  Supposedly, this problem is now solved.\n\nSecond, there is a definate shortage of parts that is somehow related\nto the six-speed Manual transmission.  So as of this posting, there is\na production hold on these cars.  She claimed part of the d

In [55]:
df2

,0
0,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...
...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...
18842,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...


In [56]:
df2.columns = ['Text'] + df2.columns.tolist()[1:]

In [57]:
df2['no_emails'] = df2['Text'].apply(lambda row: re.sub(r"\S*@\S*\s?","",row))

In [58]:
# apply(lambda row: email_tokenizer.tokenize(row['Text']),axis=1)

In [59]:
df2

,Text,no_emails
0,\n\nI am sure some bashers of Pens fans are pr...,\n\nI am sure some bashers of Pens fans are pr...
1,My brother is in the market for a high-perform...,My brother is in the market for a high-perform...
2,\n\n\n\n\tFinally you said what you dream abou...,\n\n\n\n\tFinally you said what you dream abou...
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,\nThink!\n\nIt's the SCSI card doing the DMA t...
4,1) I have an old Jasmine drive which I cann...,1) I have an old Jasmine drive which I cann...
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,DN> From: (David Nye)\nDN> A neurology\nDN> co...
18842,\nNot in isolated ground recepticles (usually ...,\nNot in isolated ground recepticles (usually ...
18843,I just installed a DX2-66 CPU in a clone mothe...,I just installed a DX2-66 CPU in a clone mothe...
18844,\nWouldn't this require a hyper-sphere. In 3-...,\nWouldn't this require a hyper-sphere. In 3-...


In [84]:
tfidf = TfidfVectorizer(min_df=0.01)

In [85]:
tfidf_train = tfidf.fit_transform(X_train)

In [86]:
df_tfidf = pd.DataFrame(tfidf_train.toarray(), columns = tfidf.get_feature_names())
df_tfidf

,00,000,10,100,11,12,13,14,15,16,...,writing,written,wrong,wrote,yeah,year,yes,yet,york,young
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.072083,0.072782,0.0,0.0,0.0,0.067474,0.0,...,0.0,0.0,0.0,0.0,0.0,0.050069,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.071272,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14129,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
14130,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
14131,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
14132,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [2]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()